In [ ]:
!pip install rdkit-pypi padelpy PyTDC -qq
!git clone https://github.com/brendaferrari/AutoPaDELPy.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [ ]:
import pandas as pd
import numpy as np
from padelpy import from_smiles
from tqdm.auto import tqdm
from tdc.single_pred import Tox
import os
from rdkit import Chem
from rdkit.Chem import AllChem

tqdm.pandas()
%cd /content/AutoPaDELPy
from functions.fingerprint_functions import FingerprintFunctions
from functions.bidimensional_functions import BidimensionalFunctions

/content/AutoPaDELPy


In [ ]:
def smiles_to_fp(smiles, radius=2, num_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, num_bits))

# UniChemDB

In [ ]:
df = pd.read_csv("/content/final-herg.csv")
df.dropna(subset = ['std_smiles'],axis = 0,inplace = True)
df.reset_index(drop = True,inplace = True)
df.reset_index()[['std_smiles','index']].to_csv('dataset.smi', sep='\t', index=False, header=False)

In [ ]:
FingerprintFunctions().do_CDK("dataset.smi")
os.rename("CDK.csv",'CDK-unichemdb.csv')
# FingerprintFunctions().do_CDKgraphonly("dataset.smi")
# FingerprintFunctions().do_CDKextended("dataset.smi")

# hERG Karim's

In [ ]:
# data = Tox(name = 'hERG_Karim')
# df = data.get_data().rename(columns = {
#     'Drug':'std_smiles',
#     'Drug_ID':'id',
#     'Y':'target',
# })
# df.reset_index()[['std_smiles','id']].to_csv('dataset.smi', sep='\t', index=False, header=False)

Found local copy...
Loading...
Done!


In [ ]:
# FingerprintFunctions().do_CDK("dataset.smi")
# os.rename("CDK.csv",'CDK-karim.csv')
# FingerprintFunctions().do_CDKgraphonly("dataset.smi")
# FingerprintFunctions().do_CDKextended("dataset.smi")

# External Datasets

In [ ]:
test_pos_df = pd.read_csv("../external_test_set_pos.csv").rename(columns = {
    'Drug':'std_smiles',
    'Drug_ID':'id',
    'Y':'target',
})
test_pos_df.reset_index()[['smiles','ACTIVITY']].to_csv('dataset.smi', sep='\t', index=False, header=False)
FingerprintFunctions().do_CDK("dataset.smi")
os.rename("CDK.csv",'CDK-external_test_set_pos.csv')

# FingerprintFunctions().do_CDKgraphonly("dataset.smi")
# FingerprintFunctions().do_CDKextended("dataset.smi")

# External Datasets- hERG 70,60

In [ ]:
h60_df = pd.read_csv("../eval_set_herg_60.csv").rename(columns = {'SMILES':'smiles'})
h60_df['target'] = h60_df.pIC50.apply(lambda x: 1 if x>=5 else 0)

In [ ]:
h60_df.reset_index()[['smiles','target']].to_csv('dataset.smi', sep='\t', index=False, header=False)
FingerprintFunctions().do_CDK("dataset.smi")
os.rename("CDK.csv",'CDK-herg60.csv')

In [ ]:
h70_df = pd.read_csv("../eval_set_herg_70.csv").rename(columns = {'SMILES':'smiles'})
h70_df['target'] = h70_df.pIC50.apply(lambda x: 1 if x>=5 else 0)

In [ ]:
h70_df.reset_index()[['smiles','target']].to_csv('dataset.smi', sep='\t', index=False, header=False)
FingerprintFunctions().do_CDK("dataset.smi")
os.rename("CDK.csv",'CDK-herg70.csv')

# Morgan


In [ ]:
def smiles_to_fp(smiles, radius=2, num_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, num_bits))

In [ ]:
df

,id,std_smiles,classes
0,CHEMBL240,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,blocks
1,CHEMBL240,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,blocks
2,CHEMBL240,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...,blocks
3,CHEMBL240,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,blocks
4,CHEMBL240,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,blocks
...,...,...,...
20404,C(CCO)CC(C(=O)O)N,N[C@H](CCCCO)C(=O)O,non-blocks
20405,RISPERIDONE,CC1=NC2=[N+](CCCC2)C(=O)C1CC[NH+]1CCC(c2noc3cc...,blocks
20406,NaN,NC(=O)NOC[C@H]1NC(=O)NC1=O,non-blocks
20407,lumefantrine,CCCC[NH+](CCCC)CC(O)c1cc(Cl)cc2c1-c1ccc(Cl)cc1...,blocks


In [ ]:
pd.concat([
    df,
    pd.DataFrame(df.std_smiles.progress_apply(smiles_to_fp).values.tolist())
],axis = 1).reset_index().rename(columns = {'index':"Name"}).drop(['std_smiles',"classes",'id'],axis = 1).to_csv("Morgan-unichem.csv",index = False)

  0%|          | 0/20409 [00:00<?, ?it/s]

[05:46:36] WARNING: not removing hydrogen atom without neighbors
[05:46:36] WARNING: not removing hydrogen atom without neighbors


In [ ]:
pd.concat([
    df,
    pd.DataFrame(df.std_smiles.progress_apply(smiles_to_fp).values.tolist())
],axis = 1).reset_index().rename(columns = {'index':"Name"}).drop(['std_smiles',"target",'id'],axis = 1).to_csv("Morgan-karim.csv",index = False)

  0%|          | 0/13445 [00:00<?, ?it/s]

In [ ]:
pd.concat([
    test_pos_df,
    pd.DataFrame(test_pos_df.smiles.progress_apply(smiles_to_fp).values.tolist())
],axis = 1).drop(['smiles'],axis = 1).rename(columns = {"ACTIVITY":'Name'}).to_csv("Morgan-external_test_set_pos.csv",index = False)


  0%|          | 0/44 [00:00<?, ?it/s]

In [ ]:
pd.concat([
    h60_df,
    pd.DataFrame(h60_df.smiles.progress_apply(smiles_to_fp).values.tolist())
],axis = 1).drop(["InChl Key","smiles","Source","pIC50"],axis = 1).rename(columns = {"target":'Name'}).to_csv("Morgan-herg60.csv",index = False)

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
pd.concat([
    h70_df,
    pd.DataFrame(h70_df.smiles.progress_apply(smiles_to_fp).values.tolist())
],axis = 1).drop(["InChl Key","smiles","Source",'PubChem CID',"pIC50"],axis = 1).rename(columns = {"target":'Name'}).to_csv("Morgan-herg70.csv",index = False)

  0%|          | 0/473 [00:00<?, ?it/s]